# GSR/GPS Pipeline Launcher 📡

Dieses Notebook erlaubt dir, **GSR-CSV** und **DB-Datei** interaktiv auszuwählen
und dann die komplette Pipeline `run_gsr_gps_pipeline` zu starten.

- Sucht Dateien in `data/raw/`
- Bietet Dropdowns zur Auswahl
- Startet die Pipeline per Button
- Zeigt die Ausgabe (Pfad zu CSV/KML/Plot etc.) unten an


In [1]:
import sys
from pathlib import Path

# Projekt-Stammverzeichnis bestimmen (eine Ebene über notebooks/)
project_root = Path('..').resolve()
sys.path.append(str(project_root))
project_root

WindowsPath('C:/Users/peter/Documents/peterspython/GSR_GPS_Shimmer')

In [2]:
from src.data_processing.gsr_gps_pipeline import run_gsr_gps_pipeline
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display, clear_output


## 1. Datenverzeichnis durchsuchen

Wir suchen alle `.csv`- und `.db`-Dateien in `data/raw/` relativ zum Projekt-Stammverzeichnis.


In [3]:
data_dir = (project_root / 'data' / 'raw').resolve()
print('Datendirectory:', data_dir)

gsr_files = sorted(data_dir.glob('*.csv')) if data_dir.exists() else []
db_files = sorted(data_dir.glob('*.db')) if data_dir.exists() else []

print('\nGefundene CSVs:')
for f in gsr_files:
    print('  -', f.name)

print('\nGefundene DBs:')
for f in db_files:
    print('  -', f.name)

if not data_dir.exists():
    print('\n⚠️ Achtung: data/raw existiert nicht. Bitte anlegen und Dateien dort ablegen.')
elif not gsr_files:
    print('\n⚠️ Keine CSV-Dateien gefunden. Bitte GSR-CSV in data/raw ablegen.')
elif not db_files:
    print('\n⚠️ Keine DB-Dateien gefunden. Bitte SQLite-DB in data/raw ablegen.')

Datendirectory: C:\Users\peter\Documents\peterspython\GSR_GPS_Shimmer\data\raw

Gefundene CSVs:
  - KIT_OST_Session1_Shimmer_C078_2025-11-07T11-46-02.csv
  - KIT_OST_Session1_Shimmer_C078_eDiary2_2025-11-06T15-16-30.csv

Gefundene DBs:
  - eDiary2_2025-11-06T15-16-30.779536Z.db
  - eDiary2_2025-11-07T11-46-02.910172Z.db


## 2. CSV & DB interaktiv auswählen

Mit den Dropdowns unten kannst du frei wählen, welche Dateien als Input dienen sollen.


In [4]:
# Dropdowns definieren (nur, wenn Dateien vorhanden sind)
gsr_dropdown = None
db_dropdown = None

if gsr_files:
    gsr_dropdown = widgets.Dropdown(
        options=[(f.name, str(f)) for f in gsr_files],
        description='GSR-CSV:',
        layout=widgets.Layout(width='500px'),
        style={'description_width': 'initial'}
    )

if db_files:
    db_dropdown = widgets.Dropdown(
        options=[(f.name, str(f)) for f in db_files],
        description='DB-Datei:',
        layout=widgets.Layout(width='500px'),
        style={'description_width': 'initial'}
    )

run_button = widgets.Button(
    description='Pipeline starten',
    button_style='success',
    icon='play'
)

output_box = widgets.Output()

def on_run_clicked(b):
    if gsr_dropdown is None or db_dropdown is None:
        with output_box:
            clear_output()
            print('⚠️ Es stehen nicht genügend Dateien zur Auswahl (CSV/DB).')
        return

    gsr_path = gsr_dropdown.value
    db_path = db_dropdown.value
    out_dir = (project_root / 'results').resolve()

    with output_box:
        clear_output()
        print('Starte Pipeline mit:')
        print('  GSR:', Path(gsr_path).name)
        print('  DB :', Path(db_path).name)
        print('\nBitte warten...')

    res = run_gsr_gps_pipeline(
        gsr_csv_path=gsr_path,
        db_path=db_path,
        out_dir=str(out_dir)
    )

    with output_box:
        print('\n✅ Pipeline fertig. Ergebnisse:')
        for k, v in res.items():
            print(k, '→', v)

run_button.on_click(on_run_clicked)

widgets_list = []
widgets_list.append(widgets.HTML('<b>Bitte GSR-CSV & DB-Datei wählen und dann die Pipeline starten:</b>'))
if gsr_dropdown is not None:
    widgets_list.append(gsr_dropdown)
if db_dropdown is not None:
    widgets_list.append(db_dropdown)
widgets_list.append(run_button)
widgets_list.append(output_box)

display(widgets.VBox(widgets_list))